In [ ]:
import numpy as np
from tensorflow.keras.models import load_model
import cv2

model_path = r"KAYITLI MODEL .h5"
model = load_model(model_path)

class_labels = ['X', 'Y', 'Z', 'M']
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def prepare_image(face):
    face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
    face = cv2.resize(face, (240, 240))
    face = face.astype("float32") / 255.0
    face = np.expand_dims(face, axis=0)
    return face

In [ ]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    predictions = []  

    for (x, y, w, h) in faces:
        margin = 30
        x1 = max(x - margin, 0)
        y1 = max(y - margin, 0)
        x2 = min(x + w + margin, frame.shape[1])
        y2 = min(y + h + margin, frame.shape[0])

        face = frame[y1:y2, x1:x2]
        if face.shape[0] == 0 or face.shape[1] == 0:
            continue

        img = prepare_image(face)
        preds = model.predict(img, verbose=0)
        pred_class = np.argmax(preds)
        confidence = np.max(preds)

        predictions.append({
            "coords": (x, y, w, h),
            "label": class_labels[pred_class] if confidence >= 0.10 else "Bilinmiyor",
            "confidence": confidence
        })

    if len(predictions) == 1:
        person = predictions[0]
        x, y, w, h = person["coords"]
        label = f"{person['label']} ({person['confidence']*100:.1f}%)"

        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, label, (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    elif len(predictions) > 1:
        for person in predictions:
            x, y, w, h = person["coords"]
            label = f"{person['label']} ({person['confidence']*100:.1f}%)"

            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, label, (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    cv2.imshow("Gercek Zamanli Yuz Tanima", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
